In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import scipy.stats as stats

# 데이터 불러오기

In [2]:
path = 'C:/Users/UserK/Desktop/Ranee/data/신용예측프로젝트/프로젝트 데이터/'

In [131]:
import warnings
from glob import glob
import os
warnings.filterwarnings('ignore')
get_ipython().run_line_magic('matplotlib', 'inline')
plt.rc('font', family='Malgun Gothic')
plt.rcParams['axes.unicode_minus'] = False

# 파일명 변경 dict
rename_dict = dict(zip(['기업정보요약', '재무제표정보', '경영진정보', '국가R&D과제', '국가R&D성과', '국가R&D연구보고서',
        '디자인권정보', '상표권정보', '신용등급정보', '실용신안정보', '온라인뉴스', '취업포탈리뷰',
        '특허정보상세', '패밀리특허정보'],
        ['comp_intro_test', 'comp_finance_test', 'comp_manager_test', 'ntis_assign_test',
        'ntis_accomp_test.csf', 'ntis_article_test', 'kisti_design_test', 'kisti_mark_test',
        'cri_test', 'kisti_utility_test', 'naver_news_test', 'jobplanet_review_test',
        'patent_detail', 'family_patent']))

# data 폴더 여부 확인 및 생성
tmp = glob(path + '*.csv')
if len(tmp)==28:
    os.makedirs(path, exist_ok=True)
    [os.rename(_, path +_[2:]) for _ in tmp]
del tmp

err_list = []
for ori_name in glob(path+'*_train.csv'):
    target = ori_name[7:-10]
    new_name = rename_dict.get(target)
    if new_name:
        new_name = ori_name.replace(target, new_name)
        os.rename(ori_name, new_name)
    elif not target.replace('_', '').replace('.', '').isalpha():
        err_list.append(ori_name)

for ori_name in glob(path+'*_test.csv'):
    target = ori_name[7:-9]
    new_name = rename_dict.get(target)
    if new_name:
        new_name = ori_name.replace(target, new_name)
        os.rename(ori_name, new_name)
    elif not target.replace('_', '').replace('.', '').isalpha():
        err_list.append(ori_name)

if err_list:
    [print(_) for _ in glob(path+'*.csv')]
else:
    del err_list, ori_name, target, new_name, rename_dict

FileNotFoundError: [WinError 3] 지정된 경로를 찾을 수 없습니다: 'C:/Users/UserK/Desktop/Ranee/data/신용예측프로젝트/프로젝트 데이터\\경영진정보_test.csv' -> 'C:/Users/UserK/Desktop/Ranee/data/신용예측프로젝트/프로젝트 데이터//Users/UserK/Desktop/Ranee/data/신용예측프로젝트/프로젝트 데이터\\경영진정보_test.csv'

- 기업정보요약 : comp_intro
- 재무제표정보 : comp_finance
- 경영진정보 : comp_manager
- 국가R&D과제 : ntis_assign (진행중인것 / 완료 --> 성과데이터로있음)
- 국가R&D성과 : ntis_accomp 
- 국가R&D연구보고서 : ntis_article
- 디자인권정보 : kisti_design
- 상표권정보 : kisti_mark
- 신용등급정보: cri
- 실용신안정보 : kisti_utility
- 온라인뉴스: naver_news
- 취업포탈리뷰 : jobplanet_review
- 특허정보상세 : patent_detail
- 패밀리특허정보 : family_patent

In [3]:
comp_manager = pd.read_csv(path +'경영진정보_train.csv')
ntis_assign = pd.read_csv(path +'국가R&D과제_train.csv')
ntis_accomp = pd.read_csv(path +'국가R&D성과_train.csv')
ntis_article = pd.read_csv(path +'국가R&D연구보고서_train.csv')
comp_intro = pd.read_csv(path +'기업정보요약_train.csv')
kisti_design = pd.read_csv(path +'디자인권정보_train.csv')
kisti_mark = pd.read_csv(path +'상표권정보_train.csv')
cri = pd.read_csv(path +'신용등급정보_train.csv')
kisti_utility = pd.read_csv(path +'실용신안정보_train.csv')
naver_news = pd.read_csv(path +'온라인뉴스_train.csv')
comp_finance = pd.read_csv(path +'재무제표정보_train.csv')
jobplanet_review = pd.read_csv(path +'취업포탈리뷰_train.csv')
patent_detail = pd.read_csv(path +'특허정보상세_train.csv')
family_patent = pd.read_csv(path +'패밀리특허정보_train.csv')

In [4]:
comp_intro.head(5)

,Unnamed: 0,BusinessNum,cmpNm,기업명,ceoNm,telNo,faxTelNo,zip,adr,dtlAdr,cmpTypNm,cmpSclNm,pbcoGbNm,indCd1,indNm
0,0,1131201292,20210903,에**,김**,02 26333232,NaN,7295.0,서울특별시 영등포구 문래로20길,"60, 5층 505호 (문래동3가,메가벤처타워)",일반,중소기업,일반기업,42321.0,일반 통신 공사업
1,1,6058609398,20220308,두원건설***,이**,043 283 4567,043 292 2002,28536.0,충청북도 청주시 청원구 중앙로 140,4층 (우암동),일반,중소기업,일반기업,41100.0,건물 건설업
2,2,1138601802,20210903,(주)드림아이***,유**,070 77666664,02 62806509,8510.0,서울 금천구 벚꽃로 298,"903호,1312호 (가산동,대륭포스트타워6차)",외감,중소기업,일반기업,62010.0,컴퓨터 프로그래밍 서비스업
3,3,7048608706,20211201,(주)드림오브***,신**,02-854-8000,050-4199-5335,3900.0,서울 마포구 마포나루길 332,"선착장 (상암동,상암나루)",일반,중소기업,일반기업,71531.0,경영컨설팅업
4,4,5088103009,20211130,(주)세***,권**,054 853 0006,054 8539929,36660.0,경북 안동시 경북대로,"389, 101호 지하",일반,중소기업,일반기업,41220.0,토목시설물 건설업


In [5]:
def dot_split(row):
    row = str(row)
    if isinstance(row, str) : 
        return row.split('.')[0]
    else:
        return row 

comp_intro['indCd1_str'] = comp_intro['indCd1'].apply(dot_split)

# 상품코드로 상품분류명

In [6]:
comp_intro.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7804 entries, 0 to 7803
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   7804 non-null   int64  
 1   BusinessNum  7804 non-null   int64  
 2   cmpNm        7804 non-null   int64  
 3   기업명          7804 non-null   object 
 4   ceoNm        7804 non-null   object 
 5   telNo        7390 non-null   object 
 6   faxTelNo     5607 non-null   object 
 7   zip          6314 non-null   float64
 8   adr          7731 non-null   object 
 9   dtlAdr       6426 non-null   object 
 10  cmpTypNm     7804 non-null   object 
 11  cmpSclNm     7804 non-null   object 
 12  pbcoGbNm     6505 non-null   object 
 13  indCd1       7789 non-null   float64
 14  indNm        7787 non-null   object 
 15  indCd1_str   7804 non-null   object 
dtypes: float64(2), int64(3), object(11)
memory usage: 975.6+ KB


In [7]:
comp_intro['indCd1_str'].unique()

array(['42321', '41100', '62010', '71531', '41220', '62021', '42320',
       '73209', '59110', '41111', '58221', '42310', '63991', '42132',
       '46522', '62090', '41229', '85501', '58211', '47912', '42110',
       '26410', '42201', '61210', '52990', '47420', '16300', '42420',
       '26429', '42311', '58222', '47910', '42131', '42491', '41121',
       '46520', '28121', '46109', '63900', '42322', '42129', '46103',
       '90191', '94990', '72111', '26120', '41129', '74100', '63100',
       '72120', '46730', '17223', '42121', '42204', '42120', '63120',
       '75900', '46491', '59114', '58200', '41221', '47312', '60229',
       '62020', '41000', '61000', '47592', '28510', '42300', '61220',
       '58220', '46599', '45219', '52991', '42412', '46510', '42499',
       '59113', '46800', '61291', '46453', '42136', '20499', '42130',
       '68112', '42134', '70100', '29299', '29162', '46595', '26420',
       '59100', '47320', '46443', '55104', '75991', '63111', '63990',
       '41119', '302

In [8]:
df_industry = pd.read_excel(path+'한국표준산업분류표.xlsx',index_col=1).reset_index().fillna(method= 'ffill')

In [9]:
df_industry.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1197 entries, 0 to 1196
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   항목명     1197 non-null   object 
 1   대분류코드   1197 non-null   object 
 2   중분류코드   1197 non-null   float64
 3   항목명.1   1197 non-null   object 
 4   소분류코드   1197 non-null   float64
 5   항목명.2   1197 non-null   object 
 6   세분류코드   1197 non-null   float64
 7   항목명.3   1197 non-null   object 
 8   세세분류코드  1197 non-null   float64
 9   항목명.4   1197 non-null   object 
dtypes: float64(4), object(6)
memory usage: 93.6+ KB


In [10]:
def namee(row) :
    return row.split('(')[0]

df_industry['항목명'] = df_industry['항목명']

In [11]:
df_industry

,항목명,대분류코드,중분류코드,항목명.1,소분류코드,항목명.2,세분류코드,항목명.3,세세분류코드,항목명.4
0,"농업, 임업 및 어업(01~03)",A,1.0,농업,11.0,작물 재배업,111.0,곡물 및 기타 식량작물 재배업,1110.0,곡물 및 기타 식량작물 재배업
1,"농업, 임업 및 어업(01~03)",A,1.0,농업,11.0,작물 재배업,112.0,"채소, 화훼작물 및 종묘 재배업",1121.0,채소작물 재배업
2,"농업, 임업 및 어업(01~03)",A,1.0,농업,11.0,작물 재배업,112.0,"채소, 화훼작물 및 종묘 재배업",1122.0,화훼작물 재배업
3,"농업, 임업 및 어업(01~03)",A,1.0,농업,11.0,작물 재배업,112.0,"채소, 화훼작물 및 종묘 재배업",1123.0,종자 및 묘목 생산업
4,"농업, 임업 및 어업(01~03)",A,1.0,농업,11.0,작물 재배업,113.0,"과실, 음료용 및 향신용 작물 재배업",1131.0,과실작물 재배업
...,...,...,...,...,...,...,...,...,...,...
1192,가구 내 고용활동 및 달리 분류되지 않은 자가 소비 생산활동(97~98),T,97.0,가구 내 고용활동,970.0,가구 내 고용활동,9700.0,가구 내 고용활동,97000.0,가구 내 고용활동
1193,가구 내 고용활동 및 달리 분류되지 않은 자가 소비 생산활동(97~98),T,98.0,달리 분류되지 않은 자가 소비를 위한 가구의 재화 및 서비스 생산활동,981.0,자가 소비를 위한 가사 생산 활동,9810.0,자가 소비를 위한 가사 생산 활동,98100.0,자가 소비를 위한 가사 생산 활동
1194,가구 내 고용활동 및 달리 분류되지 않은 자가 소비 생산활동(97~98),T,98.0,달리 분류되지 않은 자가 소비를 위한 가구의 재화 및 서비스 생산활동,982.0,자가 소비를 위한 가사 서비스 활동,9820.0,자가 소비를 위한 가사 서비스 활동,98200.0,자가 소비를 위한 가사 서비스 활동
1195,국제 및 외국기관(99),U,99.0,국제 및 외국기관,990.0,국제 및 외국기관,9900.0,국제 및 외국기관,99001.0,주한 외국 공관


In [12]:
df_industry_small = df_industry[['항목명','항목명.1','항목명.2','항목명.3','세세분류코드']].rename(columns={'항목명' : '대분류', 
                                                                                           '항목명.1' : '중분류', '항목명.2' : '소분류', 
                                                                                           '항목명.3' : '세분류','세세분류코드':'indCd1_str'})
df_industry_small['indCd1_str'] = df_industry_small['indCd1_str'].apply(dot_split)
df_industry_small

,대분류,중분류,소분류,세분류,indCd1_str
0,"농업, 임업 및 어업(01~03)",농업,작물 재배업,곡물 및 기타 식량작물 재배업,1110
1,"농업, 임업 및 어업(01~03)",농업,작물 재배업,"채소, 화훼작물 및 종묘 재배업",1121
2,"농업, 임업 및 어업(01~03)",농업,작물 재배업,"채소, 화훼작물 및 종묘 재배업",1122
3,"농업, 임업 및 어업(01~03)",농업,작물 재배업,"채소, 화훼작물 및 종묘 재배업",1123
4,"농업, 임업 및 어업(01~03)",농업,작물 재배업,"과실, 음료용 및 향신용 작물 재배업",1131
...,...,...,...,...,...
1192,가구 내 고용활동 및 달리 분류되지 않은 자가 소비 생산활동(97~98),가구 내 고용활동,가구 내 고용활동,가구 내 고용활동,97000
1193,가구 내 고용활동 및 달리 분류되지 않은 자가 소비 생산활동(97~98),달리 분류되지 않은 자가 소비를 위한 가구의 재화 및 서비스 생산활동,자가 소비를 위한 가사 생산 활동,자가 소비를 위한 가사 생산 활동,98100
1194,가구 내 고용활동 및 달리 분류되지 않은 자가 소비 생산활동(97~98),달리 분류되지 않은 자가 소비를 위한 가구의 재화 및 서비스 생산활동,자가 소비를 위한 가사 서비스 활동,자가 소비를 위한 가사 서비스 활동,98200
1195,국제 및 외국기관(99),국제 및 외국기관,국제 및 외국기관,국제 및 외국기관,99001


In [14]:
comp_intro_m1 = pd.merge(comp_intro,df_industry_small,on='indCd1_str')
comp_intro_m1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5909 entries, 0 to 5908
Data columns (total 20 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   5909 non-null   int64  
 1   BusinessNum  5909 non-null   int64  
 2   cmpNm        5909 non-null   int64  
 3   기업명          5909 non-null   object 
 4   ceoNm        5909 non-null   object 
 5   telNo        5611 non-null   object 
 6   faxTelNo     4141 non-null   object 
 7   zip          4685 non-null   float64
 8   adr          5878 non-null   object 
 9   dtlAdr       4771 non-null   object 
 10  cmpTypNm     5909 non-null   object 
 11  cmpSclNm     5909 non-null   object 
 12  pbcoGbNm     4808 non-null   object 
 13  indCd1       5909 non-null   float64
 14  indNm        5908 non-null   object 
 15  indCd1_str   5909 non-null   object 
 16  대분류          5909 non-null   object 
 17  중분류          5909 non-null   object 
 18  소분류          5909 non-null   object 
 19  세분류   

In [35]:
len(comp_intro_m1['indCd1_str'].unique())

469

In [15]:
comp_intro_m1['대분류'].value_counts()

대분류
정보통신업(58~63)                       2114
건설업(41~42)                         2061
도매 및 소매업(45~47)                     652
제조업(10~34)                          504
전문, 과학 및 기술 서비스업(70~73)             221
사업시설 관리, 사업 지원 및 임대 서비스업(74~76)      97
협회 및 단체, 수리 및 기타 개인 서비스업(94~96)      55
부동산업(68)                             47
운수 및 창고업(49~52)                      43
금융 및 보험업(64~66)                      34
숙박 및 음식점업(55~56)                     26
교육 서비스업(85)                          25
예술, 스포츠 및 여가관련 서비스업(90~91)           12
수도, 하수 및 폐기물 처리, 원료 재생업(36~39)        6
보건업 및 사회복지 서비스업(86~87)                5
전기, 가스, 증기 및 공기 조절 공급업(35)            4
공공 행정, 국방 및 사회보장 행정(84)               1
농업, 임업 및 어업(01~03)                    1
광업(05~08)                             1
Name: count, dtype: int64

In [16]:
comp_intro_m1['중분류'].value_counts()

중분류
전문직별 공사업                   1586
컴퓨터 프로그래밍, 시스템 통합 및 관리업    1098
출판업                         708
종합 건설업                      475
도매 및 상품 중개업                 456
                           ... 
공공 행정, 국방 및 사회보장 행정           1
농업                            1
항공 운송업                        1
비금속광물 광업; 연료용 제외              1
환경 정화 및 복원업                   1
Name: count, Length: 64, dtype: int64

In [17]:
comp_intro_m1.isnull().sum()

Unnamed: 0        0
BusinessNum       0
cmpNm             0
기업명               0
ceoNm             0
telNo           298
faxTelNo       1768
zip            1224
adr              31
dtlAdr         1138
cmpTypNm          0
cmpSclNm          0
pbcoGbNm       1101
indCd1            0
indNm             1
indCd1_str        0
대분류               0
중분류               0
소분류               0
세분류               0
dtype: int64

In [127]:
comp_intro_m1.loc[comp_intro_m1['indNm'].isnull()]

,Unnamed: 0,BusinessNum,cmpNm,기업명,ceoNm,telNo,faxTelNo,zip,adr,dtlAdr,cmpTypNm,cmpSclNm,pbcoGbNm,indCd1,indNm,indCd1_str,대분류,중분류,소분류,세분류
1551,728,4060500072,20220308,우*,최**,NaN,NaN,11942.0,경기 구리시 이문안로129번길 30,307호 (수택동),일반,중소기업,일반기업,41111.0,NaN,41111,건설업,종합 건설업,건물 건설업,주거용 건물 건설업


In [ ]:
comp_intro_m1.loc[df_m1['indNm'].isnull()]

In [128]:
comp_intro_m1['cmpSclNm'].value_counts()

cmpSclNm
중소기업         5586
중견기업          180
대기업            81
기타(비영리법인)      58
중견기업(하)         2
대기업(금융)         2
Name: count, dtype: int64

In [133]:
comp_intro_m1['pbcoGbNm'].value_counts()

pbcoGbNm
일반기업    4804
공기업        4
Name: count, dtype: int64

In [145]:
comp_intro_m1.loc[comp_intro_m1['cmpSclNm'] == '기타(비영리법인)']

,Unnamed: 0,BusinessNum,cmpNm,기업명,ceoNm,telNo,faxTelNo,zip,adr,dtlAdr,cmpTypNm,cmpSclNm,pbcoGbNm,indCd1,indNm,indCd1_str,대분류,중분류,소분류,세분류
1050,275,1088209536,20221022,현대산업정보***,최**,02-3143-0041,NaN,NaN,서울 마포구 월드컵로28길 20 (성산동),NaN,일반,기타(비영리법인),NaN,71531.0,경영컨설팅업,71531,"전문, 과학 및 기술 서비스업",전문 서비스업,회사 본부 및 경영 컨설팅 서비스업,경영 컨설팅 및 공공 관계 서비스업
3573,2202,6158103807,20211229,(주)장유정***,박**,055 331 7456,055 331 7457,51008.0,경남 김해시 장유로329번길 36-17,1층 (신문동),일반,기타(비영리법인),일반기업,42322.0,내부 통신배선 공사업,42322,건설업,전문직별 공사업,전기 및 통신 공사업,통신 공사업
3710,82,8728203394,20221022,한국저작권***,박**,02-1588-0190,NaN,NaN,"서울특별시 마포구 월드컵북로 400 9층 (상암동,서울산업진흥원빌딩)",NaN,일반,기타(비영리법인),NaN,94990.0,그 외 기타 협회 및 단체,94990,"협회 및 단체, 수리 및 기타 개인 서비스업",협회 및 단체,기타 협회 및 단체,그 외 기타 협회 및 단체
3711,313,1198209415,20210908,(사)한국화학안***,지**,031-410-2991,031-410-2994,7623.0,서울 강서구 공항대로,"36, 302호 (공항동)",일반,기타(비영리법인),일반기업,94990.0,그 외 기타 협회 및 단체,94990,"협회 및 단체, 수리 및 기타 개인 서비스업",협회 및 단체,기타 협회 및 단체,그 외 기타 협회 및 단체
3712,523,1058209828,20221022,국제개발협력민간***,오**,02-2279-1705,NaN,NaN,"서울 마포구 독막로 282 4층 (대흥동,대흥빌딩)",NaN,일반,기타(비영리법인),NaN,94990.0,그 외 기타 협회 및 단체,94990,"협회 및 단체, 수리 및 기타 개인 서비스업",협회 및 단체,기타 협회 및 단체,그 외 기타 협회 및 단체
3713,699,2768207917,20221022,컨선월드와이***,이준모외***,02-324-3900,NaN,NaN,"서울 마포구 양화로11길 12 5층 (서교동,현담빌딩)",NaN,일반,기타(비영리법인),NaN,94990.0,그 외 기타 협회 및 단체,94990,"협회 및 단체, 수리 및 기타 개인 서비스업",협회 및 단체,기타 협회 및 단체,그 외 기타 협회 및 단체
3714,1022,1248209725,20221022,인권재***,박**,02-363-5855,NaN,NaN,"서울특별시 은평구 증산로 403-1 2층 (신사동,복지빌딩)",NaN,일반,기타(비영리법인),NaN,94990.0,그 외 기타 협회 및 단체,94990,"협회 및 단체, 수리 및 기타 개인 서비스업",협회 및 단체,기타 협회 및 단체,그 외 기타 협회 및 단체
3715,1140,2068209956,20221022,어린이와작은도서***,백**,02-388-5933,NaN,NaN,서울 마포구 월드컵북로5가길 8-7 (서교동),NaN,일반,기타(비영리법인),NaN,94990.0,그 외 기타 협회 및 단체,94990,"협회 및 단체, 수리 및 기타 개인 서비스업",협회 및 단체,기타 협회 및 단체,그 외 기타 협회 및 단체
3716,1512,1168206394,20211228,(사)한국정보산업***,정**,02 780 0201,02 782 1266,6221.0,서울 강남구 테헤란로 222,"3층, 4층 (역삼동,도원빌딩)",일반,기타(비영리법인),일반기업,94990.0,그 외 기타 협회 및 단체,94990,"협회 및 단체, 수리 및 기타 개인 서비스업",협회 및 단체,기타 협회 및 단체,그 외 기타 협회 및 단체
3717,2010,8588207928,20221022,남북보건의료교***,배**,02-712-5587,NaN,NaN,"서울특별시 마포구 큰우물로 76 413호 (도화동,고려빌딩)",NaN,일반,기타(비영리법인),NaN,94990.0,그 외 기타 협회 및 단체,94990,"협회 및 단체, 수리 및 기타 개인 서비스업",협회 및 단체,기타 협회 및 단체,그 외 기타 협회 및 단체


In [146]:
comp_intro_m1.loc[comp_intro_m1['cmpSclNm'] == '대기업(금융)']

,Unnamed: 0,BusinessNum,cmpNm,기업명,ceoNm,telNo,faxTelNo,zip,adr,dtlAdr,cmpTypNm,cmpSclNm,pbcoGbNm,indCd1,indNm,indCd1_str,대분류,중분류,소분류,세분류
4540,373,2068103702,20210818,케이씨에이손해사정***,허**,02 450 2973,02 450 2975,4143.0,서울 마포구 마포대로 137,(공덕동),외감,대기업(금융),일반기업,66201.0,손해사정업,66201,금융 및 보험업,금융 및 보험관련 서비스업,보험 및 연금관련 서비스업,보험 및 연금관련 서비스업
5817,5245,2208702750,20210908,카카오페이증권***,김대홍외***,16008515,02 60998519,13529.0,경기 성남시 분당구 판교역로146번길 20,"12층 (백현동,오피스에이치)",외감,대기업(금융),일반기업,66121.0,증권 중개업,66121,금융 및 보험업,금융 및 보험관련 서비스업,금융 지원 서비스업,증권 및 선물 중개업


In [18]:
comp_intro_m1.loc[comp_intro_m1['pbcoGbNm'] == '공기업']

,Unnamed: 0,BusinessNum,cmpNm,기업명,ceoNm,telNo,faxTelNo,zip,adr,dtlAdr,cmpTypNm,cmpSclNm,pbcoGbNm,indCd1,indNm,indCd1_str,대분류,중분류,소분류,세분류
3722,3642,2098206974,20220128,영상물등급***,채**,02 31534300,02 31534337,48058.0,부산 해운대구 센텀서로 39,(우동),일반,기타(비영리법인),공기업,94990.0,그 외 기타 협회 및 단체,94990,"협회 및 단체, 수리 및 기타 개인 서비스업(94~96)",협회 및 단체,기타 협회 및 단체,그 외 기타 협회 및 단체
5286,1088,1058201494,20211201,(학)서강***,박**,02 705 8114,02 705 8147,4107.0,서울 마포구 백범로 35,(신수동),일반,기타(비영리법인),공기업,85302.0,대학교,85302,교육 서비스업(85),교육 서비스업,고등 교육기관,고등 교육기관
5288,5300,1058200553,20211201,서강대**,박**,02 705 8114,02-718-2065,4107.0,서울 마포구 백범로 35,(신수동),일반,기타(비영리법인),공기업,85302.0,대학교,85302,교육 서비스업(85),교육 서비스업,고등 교육기관,고등 교육기관
5628,2440,6148200106,20211228,정보통신기획***,석**,042-612-8001,042-612-8809,34054.0,대전 유성구 유성대로 1548,(화암동),일반,기타(비영리법인),공기업,70209.0,기타 인문 및 사회과학 연구개발업,70209,"전문, 과학 및 기술 서비스업(70~73)",연구개발업,인문 및 사회과학 연구개발업,인문 및 사회과학 연구개발업


In [135]:
comp_intro_m1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27593 entries, 0 to 27592
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             27593 non-null  int64  
 1   BusinessNum            27593 non-null  int64  
 2   SitId                  6821 non-null   object 
 3   ProjectName            5548 non-null   object 
 4   ProjectCode            5548 non-null   float64
 5   EquipId                6821 non-null   float64
 6   EquipNo                6821 non-null   object 
 7   Year                   6821 non-null   float64
 8   ProjectYear            5548 non-null   float64
 9   MinistryCode           5548 non-null   object 
 10  MinistryName           5548 non-null   object 
 11  BudgetProjectNo        5548 non-null   object 
 12  BudgetName             6821 non-null   object 
 13  PerformAgentCode       5544 non-null   object 
 14  PerformAgent           5548 non-null   object 
 15  6T

In [20]:
comp_finance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 888634 entries, 0 to 888633
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Unnamed: 0   888634 non-null  int64  
 1   BusinessNum  888634 non-null  int64  
 2   stYear       887950 non-null  float64
 3   stNm         887950 non-null  object 
 4   acctCd       887950 non-null  object 
 5   accNm        887950 non-null  object 
 6   acctAmt      887947 non-null  float64
dtypes: float64(2), int64(2), object(3)
memory usage: 47.5+ MB


In [21]:
comp_finance['accNm'].unique()

array([nan, '유동자산', '당좌자산', '매출채권', '재고자산', '투자자산', '자산총계', '유동부채',
       '비유동부채', '부채총계', '이익잉여금', '자본총계', '매출액', '매출원가', '매출총이익',
       '판매비와관리비', '인건비', '영업이익', '영업외비용', '법인세비용', '당기순이익(손실)', '총자산 증가율',
       '유형자산 증가율', '매출 증가율', '총자산', '자기자본', '차입금', '유동비율', '대손상각비',
       '이자비용', '노무비', '제조경비', '제조원가', '원재료', '자본잉여금', '수출액1', '수출액2',
       '수출액3', '수출액5', '당좌부채(당좌차월)', '수출액4'], dtype=object)

In [22]:
comp_finance['accNm'].value_counts()

accNm
총자산 증가율       31571
유형자산 증가율      31571
매출 증가율        31571
자기자본          31570
총자산           31570
유동비율          31551
자산총계          31337
유동자산          31335
자본총계          31333
영업이익          31327
당기순이익(손실)     31327
부채총계          31302
유동부채          31293
당좌자산          31211
판매비와관리비       31201
차입금           31154
매출총이익         31022
매출액           31019
인건비           30792
이익잉여금         30746
영업외비용         30060
매출채권          27227
매출원가          26415
이자비용          25023
투자자산          22447
비유동부채         21927
법인세비용         18381
재고자산          17731
제조경비          17525
노무비           15919
제조원가          14616
대손상각비         11003
원재료            8872
자본잉여금          4354
수출액3            381
수출액2            132
수출액1             70
당좌부채(당좌차월)       56
수출액4              5
수출액5              3
Name: count, dtype: int64

In [23]:
pv1 = comp_finance.pivot_table(index=['BusinessNum','accNm'] , values='acctAmt', aggfunc='mean').reset_index()
df_sales = pv1.loc[pv1['accNm'] == '매출액']

In [24]:
comp_intro_m2 = pd.merge(comp_intro_m1, df_sales, on='BusinessNum')

In [29]:
# 귀무가설 : 정규성을 따른다 
# 대립가설 : 정규성을 따르지 않는다.
stats.normaltest(comp_intro_m2['acctAmt']) # < 0.05 -> 귀무가설 기각

NormaltestResult(statistic=10592.817202765118, pvalue=0.0)

In [32]:
# 귀무가설 : 대분류별 중앙값 차이있다
# 귀무가설 : 대분류별 중앙값 차이없다
grouped_data = [comp_intro_m2[comp_intro_m2['대분류'] == group]['acctAmt'] for group in comp_intro_m2['대분류'].unique()]
stats.kruskal(*grouped_data)

KruskalResult(statistic=244.18752631970986, pvalue=3.2036747297560524e-42)

In [19]:
ntis_assign.head(5)

,Unnamed: 0,BusinessNum,ProjectNo,ProjectNameKR,ProjectNameEN,ManagerName,ResearchersName,ResManCount,ResWmanCount,GoalFull,...,TechLifecycle,RegionCode,Region,EconomicSocialGoal,SixtechCode,Sixtech,ApplyareaFirstCode,ApplyareaFirst,ApplyareaSecond,ApplyareaThird
0,0,1051505526,1.425154e+09,원자력용 부품의 형상 가공 및 캠 방식 progressive 기술 개발,Develpment of Shape processing and CAM-based p...,김영남,['정동진'],2.0,NaN,0 원자력용 부품의 형상 가공 및 캠 방식 progressive 기술 개발 ...,...,성장기,8,경기도 광주시,NaN,70000,위의 미래유망신기술(6T) 103개 세분류에 속하지 않는 기타 연구,Y10,제조업(전기 및 기계장비),0,49760000
1,1,1051505526,1.425141e+09,스위치용 회전부품의 균일한 tension 특성 구현 기술,NaN,조을훈,"['김영남', '정동진']",3.0,NaN,0 VCB 보조 스위치용 회전단자의 균일한 tension 특성 구현 기술 -...,...,성장기,8,경기도 광주시,NaN,70000,위의 미래유망신기술(6T) 103개 세분류에 속하지 않는 기타 연구,Y07,제조업(비금속광물 및 금속제품),0,50000000
2,2,1051505526,1.425136e+09,수동 운전시 강하 속도제어 기능의 수문권양기 개발,NaN,박세진,"['김범중', '이준형']",3.0,NaN,수동 강하제어가 가능한 수문권양기의 개발 필요성 자중강하 방식의 수문권양기는 수문의...,...,성장기,12,전라북도 군산시,NaN,70000,위의 미래유망신기술(6T) 103개 세분류에 속하지 않는 기타 연구,Y10,제조업(전기 및 기계장비),0,25000000
3,3,1051505526,1.415125e+09,제진기능 향상을 위한 Multi level Automatic 제진기,NaN,이광호,"['박세진', '김범중', '채수현', '이환희', '김재동', '김성진', '서영수']",8.0,NaN,유압식과 로타리식의 장점을 활용하여 레이크가 설치된 프레임이 좌우로 이동 및 상하로...,...,성숙기,12,전라북도,NaN,70000,위의 미래유망신기술(6T) 103개 세분류에 속하지 않는 기타 연구,Y13,"하수,폐기물처리, 원료재생 및 환경복원업",3,142000000
4,4,1051505526,1.425154e+09,원자력용 부품의 형상 가공 및 캠 방식 progressive 기술 개발,Develpment of Shape processing and CAM-based p...,김영남,['정동진'],2.0,NaN,0 원자력용 부품의 형상 가공 및 캠 방식 progressive 기술 개발 ...,...,성장기,8,경기도 광주시,NaN,70000,위의 미래유망신기술(6T) 103개 세분류에 속하지 않는 기타 연구,Y10,제조업(전기 및 기계장비),0,49760000


In [33]:
comp_finance

,Unnamed: 0,BusinessNum,stYear,stNm,acctCd,accNm,acctAmt
0,0,1051505526,NaN,NaN,NaN,NaN,NaN
1,1,2118605035,2017.0,재무상태표,M11000,유동자산,9440422.000
2,2,2118605035,2017.0,재무상태표,M11100,당좌자산,9358829.000
3,3,2118605035,2017.0,재무상태표,M11130,매출채권,1934474.000
4,4,2118605035,2017.0,재무상태표,M11500,재고자산,81593.000
...,...,...,...,...,...,...,...
888629,888629,6138108916,2021.0,성장성지표,MA1600,매출 증가율,-4.190
888630,888630,6138108916,2021.0,성장성지표,MA5100,총자산,3618115.000
888631,888631,6138108916,2021.0,성장성지표,MA5300,자기자본,2511720.000
888632,888632,6138108916,2021.0,성장성지표,MA5900,차입금,0.000


In [34]:
ntis_assign.head(10)

,Unnamed: 0,BusinessNum,ProjectNo,ProjectNameKR,ProjectNameEN,ManagerName,ResearchersName,ResManCount,ResWmanCount,GoalFull,...,TechLifecycle,RegionCode,Region,EconomicSocialGoal,SixtechCode,Sixtech,ApplyareaFirstCode,ApplyareaFirst,ApplyareaSecond,ApplyareaThird
0,0,1051505526,1.425154e+09,원자력용 부품의 형상 가공 및 캠 방식 progressive 기술 개발,Develpment of Shape processing and CAM-based p...,김영남,['정동진'],2.0,NaN,0 원자력용 부품의 형상 가공 및 캠 방식 progressive 기술 개발 ...,...,성장기,8,경기도 광주시,NaN,70000,위의 미래유망신기술(6T) 103개 세분류에 속하지 않는 기타 연구,Y10,제조업(전기 및 기계장비),0,49760000
1,1,1051505526,1.425141e+09,스위치용 회전부품의 균일한 tension 특성 구현 기술,NaN,조을훈,"['김영남', '정동진']",3.0,NaN,0 VCB 보조 스위치용 회전단자의 균일한 tension 특성 구현 기술 -...,...,성장기,8,경기도 광주시,NaN,70000,위의 미래유망신기술(6T) 103개 세분류에 속하지 않는 기타 연구,Y07,제조업(비금속광물 및 금속제품),0,50000000
2,2,1051505526,1.425136e+09,수동 운전시 강하 속도제어 기능의 수문권양기 개발,NaN,박세진,"['김범중', '이준형']",3.0,NaN,수동 강하제어가 가능한 수문권양기의 개발 필요성 자중강하 방식의 수문권양기는 수문의...,...,성장기,12,전라북도 군산시,NaN,70000,위의 미래유망신기술(6T) 103개 세분류에 속하지 않는 기타 연구,Y10,제조업(전기 및 기계장비),0,25000000
3,3,1051505526,1.415125e+09,제진기능 향상을 위한 Multi level Automatic 제진기,NaN,이광호,"['박세진', '김범중', '채수현', '이환희', '김재동', '김성진', '서영수']",8.0,NaN,유압식과 로타리식의 장점을 활용하여 레이크가 설치된 프레임이 좌우로 이동 및 상하로...,...,성숙기,12,전라북도,NaN,70000,위의 미래유망신기술(6T) 103개 세분류에 속하지 않는 기타 연구,Y13,"하수,폐기물처리, 원료재생 및 환경복원업",3,142000000
4,4,1051505526,1.425154e+09,원자력용 부품의 형상 가공 및 캠 방식 progressive 기술 개발,Develpment of Shape processing and CAM-based p...,김영남,['정동진'],2.0,NaN,0 원자력용 부품의 형상 가공 및 캠 방식 progressive 기술 개발 ...,...,성장기,8,경기도 광주시,NaN,70000,위의 미래유망신기술(6T) 103개 세분류에 속하지 않는 기타 연구,Y10,제조업(전기 및 기계장비),0,49760000
5,5,1051505526,1.425098e+09,인덱스 및 다축을 활용한 multi-tap 가공 기술개발,NaN,조규종,"['고현옥', '김정균', '조을훈', '신귀분']",3.0,2.0,○ multi-tap 반자동화 가공 system 설계 및 개발 - 최적의 공...,...,성장기,8,경기도,NaN,70000,위의 미래유망신기술(6T) 103개 세분류에 속하지 않는 기타 연구,Y10,제조업(전기 및 기계장비),1,45000000
6,6,1051505526,1.410051e+09,항균용 은나노 세라믹 복합체 개발,NaN,이형우,NaN,NaN,NaN,"은나노입자 및 그라스비드의 제조 및 실험 설계-은나노입자-그라스비드 복합체의 제조,...",...,성숙기,3,대구광역시,산업생산 및 기술,30000,NT분야,NaN,NaN,0,134000000
7,7,1051505526,1.415120e+09,항당뇨 강화 기능성 엽채류 개발 및 상품화,NaN,이무상,NaN,5.0,3.0,"본 연구는 2011년 지역산업기술개발사업을 통해 무균, 무공해에다 인체에 꼭 필요한...",...,도입기,3,대구광역시,농업생산 및 기술,20215,기능성 바이오소재 기반기술,Y01,"농업, 임업 및 어업",0,150000000
8,8,1051505526,1.425099e+09,조명 제어용 파워릴레이의 접촉 내구성(수명) 향상기술개발,NaN,조을훈,"['정재성', '박진주', '김현식', '고현옥', '김경훈', '소형섭', '이선...",10.0,4.0,파워 릴레이의 동작 구조 개선 설계 및 시제품 개발 - 좌우 방식의 동작 구조...,...,성장기,8,경기도,NaN,70000,위의 미래유망신기술(6T) 103개 세분류에 속하지 않는 기타 연구,Y10,제조업(전기 및 기계장비),0,297000000
9,9,1051505526,1.425141e+09,스위치용 회전부품의 균일한 tension 특성 구현 기술,NaN,조을훈,"['김영남', '정동진']",3.0,NaN,0 VCB 보조 스위치용 회전단자의 균일한 tension 특성 구현 기술 -...,...,성장기,8,경기도 광주시,NaN,70000,위의 미래유망신기술(6T) 103개 세분류에 속하지 않는 기타 연구,Y07,제조업(비금속광물 및 금속제품),0,50000000
